In [2]:
import os
from huggingface_hub import InferenceClient
os.environ.get("HUGGING_FACE_API_KEY")

In [14]:
#client = InferenceClient("meta-llama/Llama-3.3-70B-Instruct")
#client=InferenceClient("https://jc26mwg228mkj8dw.us-east-1.aws.endpoints.huggingface.cloud")
#client = InferenceClient(provider="hf-inference", model="meta-llama/Llama-3.3-70B-Instruct")
client = InferenceClient("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [ ]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of France is"},
    ],
    stream=False,
    max_tokens=1024,
)
print(output.choices[0].message.content)

In [17]:
SYSTEM_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json blob.
Specifically, this json should have an `action` key (with the name of the tool to use) and an `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are:
get_weather: Get the current weather in a given location, args: {"location": {"type": "string"}}
example use :

{{
  "action": "get_weather",
  "action_input": {"location": "New York"}
}}


ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:
Action:

$JSON_BLOB (inside markdown cell)

Observation: the result of the action. This Observation is unique, complete, and the source of truth.
... (this Thought/Action/Observation can repeat N times, you should take several steps when needed. The $JSON_BLOB must be formatted as markdown and only use a SINGLE action at a time.)

You must always end your output with the following format:

Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. """

In [18]:
prompt=f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{SYSTEM_PROMPT}
<|eot_id|><|start_header_id|>user<|end_header_id|>
What's the weather in London ?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [21]:
messages=[
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What's the weather in London ?"},
    ]
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

tokenizer.apply_chat_template(messages, tokenize=False,add_generation_prompt=True)

'<s> [INST] Answer the following questions as best you can. You have access to the following tools:\n\nget_weather: Get the current weather in a given location\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have an `action` key (with the name of the tool to use) and an `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are:\nget_weather: Get the current weather in a given location, args: {"location": {"type": "string"}}\nexample use :\n\n{{\n  "action": "get_weather",\n  "action_input": {"location": "New York"}\n}}\n\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about one action to take. Only one action at a time in this format:\nAction:\n\n$JSON_BLOB (inside markdown cell)\n\nObservation: the result of the action. This Observation is unique, complete, and the source of truth.\n... (this Thought/Action/Obser

In [ ]:
output = client.text_generation(
    prompt,
    max_new_tokens=200,
)
print(output)


- At this point the model is hallucinating, becaue it's producing a fabricated "Observation" - a response that it generates on it's own rather than being the result of an actual function or tool call. To prevent this, we stop generating right before "Observation". This allows us to manually run the function and then insert the real output as the Observation.